<a href="https://colab.research.google.com/github/avs20/SioLabsPython0/blob/main/Amazon_Fine_Food_Reviews_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/snap/amazon-fine-food-reviews

[EDA](https://nycdatascience.com/blog/student-works/amazon-fine-foods-visualization/)

## 1.Load the dataset from google drive

In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
import os
os.chdir('/content/drive/MyDrive/SioLabs/dataset')
!ls -lh

total 2.2G
-rw------- 1 root root 356M Feb  9 09:21 database.sqlite
-r-------- 1 root root 1.6G Sep  4  2016 GoogleNews-vectors-negative300.bin.gz
-rw------- 1 root root 287M Sep 18  2019 Reviews.csv


In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline
import sklearn

In [4]:
data = pd.read_csv('Reviews.csv')
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [82]:
data.shape

(568454, 11)

In [7]:
def partition(x):
  if x < 3 :
    return 0 
  return 1


data['Sentiment'] = data['Score'].apply(partition)
print(data['Sentiment'].shape)
print(data.head(3))

(568454,)
   Id   ProductId  ...                                               Text Sentiment
0   1  B001E4KFG0  ...  I have bought several of the Vitality canned d...         1
1   2  B00813GRG4  ...  Product arrived labeled as Jumbo Salted Peanut...         0
2   3  B000LQOCH0  ...  This is a confection that has been around a fe...         1

[3 rows x 11 columns]


In [29]:
data[data.duplicated(subset = [ 'ProductId', 'ProfileName', 'Score', 'Time', 'Summary'], keep = False)]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
466,467,B000G6RYNE,A3PJZ8TU8FDQ1K,Jared Castle,0,0,5,1231718400,"One bite and you'll become a ""chippoisseur""","I'm addicted to salty and tangy flavors, so wh...",1
574,575,B000G6RYNE,A3PJZ8TU8FDQ1K,Jared Castle,2,2,5,1231718400,"One bite and you'll become a ""chippoisseur""","I'm addicted to salty and tangy flavors, so wh...",1
1084,1085,B0030N5K1I,A19U4T4I7UZGNM,3 dogs,3,3,1,1330992000,awful,"This is not jerky, this is processed, hard lik...",0
1087,1088,B0030N5K1I,A19U4T4I7UZGNM,3 dogs,1,1,1,1330992000,awful,"This is not jerky, this is processed, hard lik...",0
2630,2631,B0016FY6H6,A2NLZ3M0OJV9NX,Mark Bodzin,1,1,2,1313107200,Way too weak for my tastes,"What can i say, I love iced tea. I drink a lot...",0
...,...,...,...,...,...,...,...,...,...,...,...
568007,568008,B0030VJ8YU,A20P8VC55KPPCT,FL Mom,3,3,5,1274486400,These pouches are great!,These pouches are so easy and convenient. They...,1
568008,568009,B0030VJ8YU,ATIHDHZYNQ0EI,"Kristen O'donnell ""twinsmom""",3,3,5,1270425600,Organic and Tasty!,I have 12 month olds and no time to write a gr...,1
568017,568018,B0030VJ8YU,A20P8VC55KPPCT,FL Mom,2,2,5,1274486400,These pouches are great!,These pouches are so easy and convenient. They...,1
568034,568035,B0030VJ8YU,A1S6HK2R1KUC5E,data1701,1,1,5,1318291200,Triplets love this!,This is very smooth and easy to feed to my tri...,1


In [30]:
data[data.duplicated(subset = [ 'ProductId', 'ProfileName', 'Score', 'Time', 'Summary'], keep = False)].loc[data.ProductId == 'B005K4Q1VI', :]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
3306,3307,B005K4Q1VI,A3OXHLG6DIBRW8,"C. F. Hill ""CFH""",24,25,5,1321401600,Really Good Hot Cocoa - Highly Recommended,These Grove Square Hot Cocoa flavors are by fa...,1
3416,3417,B005K4Q1VI,A3OXHLG6DIBRW8,"C. F. Hill ""CFH""",5,5,5,1321401600,Really Good Hot Cocoa - Highly Recommended,These Grove Square Hot Cocoa flavors are by fa...,1


[Click Here to look into 1 product description](https://www.amazon.com/dp/B005K4Q1VI)

## Data Cleaning 

1. Remove Duplicates

In [31]:
#Sorting data according to ProductId in ascending order
sorted_data=data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [32]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(393933, 11)

In [33]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(data['Id'].size*1.0)*100

69.29901100176971

Checking some constraints like helpfullness numerator should be greater than denominator. So checking on them 



In [34]:
final[final['HelpfulnessNumerator'] > final['HelpfulnessDenominator']]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
64421,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...,1
44736,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...,1


Need to drop these 2 rows 

In [57]:
final = final[final['HelpfulnessNumerator'] <= final['HelpfulnessDenominator']]
final.reset_index(inplace=True)
final.shape

(393931, 12)

## Preprocessing

In [58]:
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/

import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os


# https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
from bs4 import BeautifulSoup


In [59]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [60]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [61]:
# Combining all the above steps 
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|██████████| 393931/393931 [02:56<00:00, 2235.33it/s]


In [62]:
preprocessed_reviews[1]

'fun way children learn months year learn poems throughout school year like handmotions invent poem'

In [63]:
preprocessed_reviews[15000]

'catnip cute usable size larger expected catnip came clear plastic bag tied opening inside good amount cats went crazy'

In [64]:
len(preprocessed_reviews)

393931

## Featurization
Convert the review to vectors

In [65]:
sample_data = final.sample(n = 5000)
sample_data.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
197000,94274,94275,B001EPQ18K,A3LWGDI3CYQML0,Amazon Shopper2,0,0,4,1323993600,Excellent cookies,"The lemon is not overpowering, though I wish i...",1
142469,106128,106129,B000YHNQSC,ADCQFR0DZQC7A,flygirl,0,0,5,1292803200,jackie,just received this item so glad I ordered it b...,1
34702,420893,420894,B000B9EB4Q,AOY13ZY2UCBT0,Kevin,1,1,1,1273104000,"Weak, poor quality","This is a very, very mild ginger tea. I take ...",0
79593,173810,173811,B000GAO91K,A2EDE0QF9MSCTW,VVL84,1,1,5,1350432000,DOES A BODY GOOD,I AM CURRENTLY TRYING TO LOOSE SOME WEIGHT SO ...,1
141048,100663,100664,B000XKCDVG,ADUNITSKC9LS5,"putty butt ""Keri Starkel""",0,0,5,1318291200,Great dog food!!!!!,"I was told from a friend of mine, how great bl...",1


In [66]:
sample_data.drop(columns = ['Id', 'ProductId', 'UserId','ProfileName', 'Time', 'Summary', 'Score'], inplace=True)

In [67]:
sample_data.head(2)

,index,HelpfulnessNumerator,HelpfulnessDenominator,Text,Sentiment
197000,94274,0,0,"The lemon is not overpowering, though I wish i...",1
142469,106128,0,0,just received this item so glad I ordered it b...,1


In [68]:
sample_data.index.values

array([197000, 142469,  34702, ..., 125648,  61047, 208724])

In [70]:
sample_reviews = [ preprocessed_reviews[i] for i in sample_data.index.values]
sample_reviews[0]

'lemon not overpowering though wish bit tartness would known missing something good would purchased sooner good thing bought boxes'

In [85]:
sample_data['preprocessed'] = sample_reviews
sample_data.head()

,index,HelpfulnessNumerator,HelpfulnessDenominator,Text,Sentiment,preprocessed
197000,94274,0,0,"The lemon is not overpowering, though I wish i...",1,lemon not overpowering though wish bit tartnes...
142469,106128,0,0,just received this item so glad I ordered it b...,1,received item glad ordered skeptical spend kin...
34702,420893,1,1,"This is a very, very mild ginger tea. I take ...",0,mild ginger tea take least bags water even tas...
79593,173810,1,1,I AM CURRENTLY TRYING TO LOOSE SOME WEIGHT SO ...,1,currently trying loose weight looking substitu...
141048,100663,0,0,"I was told from a friend of mine, how great bl...",1,told friend mine great blue buffalo dog food k...


In [87]:
sample_data.drop(columns = ['Text', 'index'], inplace=True)

In [88]:
sample_data.head(3)

,HelpfulnessNumerator,HelpfulnessDenominator,Sentiment,preprocessed
197000,0,0,1,lemon not overpowering though wish bit tartnes...
142469,0,0,1,received item glad ordered skeptical spend kin...
34702,1,1,0,mild ginger tea take least bags water even tas...


In [84]:
from sklearn.model_selection import train_test_split

In [89]:
y = sample_data['Sentiment'].values 
X = sample_data.drop(columns =['Sentiment'])

print(X.shape)
print(y.shape)

(5000, 3)
(5000,)


In [105]:
sample_data['Sentiment'].value_counts()

1    4292
0     708
Name: Sentiment, dtype: int64

In [90]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 24)

print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

(4000, 3)
(4000,)
(1000, 3)
(1000,)


### BOW

In [91]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect.fit(train_X['preprocessed'])

print("some feature names ", count_vect.get_feature_names()[:10])
print('='*50)

some feature names  ['aaa', 'ab', 'abandoned', 'abc', 'abcs', 'abd', 'ability', 'able', 'abnormalities', 'abomination']


In [92]:
final_vectors = count_vect.transform(train_X['preprocessed'])

In [93]:
final_vectors.shape

(4000, 13210)

In [95]:
num_feats = train_X[['HelpfulnessNumerator' ,	'HelpfulnessDenominator']].values

from scipy import sparse

training_data = sparse.hstack(( num_feats, final_vectors))

training_data.shape

(4000, 13212)

In [96]:
from sklearn.linear_model import LogisticRegression

In [100]:
model = LogisticRegression(max_iter = 15000)

model.fit(training_data, train_y)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=15000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [101]:
# predict on test datasets 

final_test_vectors = count_vect.transform(test_X['preprocessed'].values)
final_test_vectors.shape

(1000, 13210)

In [102]:
test_feats = test_X[['HelpfulnessNumerator' ,	'HelpfulnessDenominator']].values

test_data = sparse.hstack(( test_feats, final_test_vectors))

test_data.shape

(1000, 13212)

In [103]:
from sklearn.metrics import accuracy_score

preds = model.predict(test_data)

# get the actual values 
y_true = test_y

accuracy_score(y_true, preds)

0.889